In [35]:
import math
import copy
import numpy as np
import matplotlib.pyplot as plt


# Training Dataset

| Size (sqft) | Number of Bedrooms  | Number of floors | Age of  Home | Price (1000s dollars)  |   
| ----------------| ------------------- |----------------- |--------------|-------------- |  
| 2104            | 5                   | 1                | 45           | 460           |  
| 1416            | 3                   | 2                | 40           | 232           |  
| 852             | 2                   | 1                | 35           | 178           |  

In [36]:
X_train = np.array([
    [2104, 5, 1, 45],
    [1416, 3, 2, 40],
    [852, 2, 1, 35]
])
y_train = np.array([460, 232, 178])
print(X_train.shape)

(3, 4)


In [37]:
b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])
# print(f"w_init shape: {w_init.shape}, b_init type: {type(b_init)}")

In [38]:
def predict_single_loop(x: np.ndarray, w: np.ndarray, b: float):
    """
    Predicts output for a training input of multiple linear regression using looping method
    Args:
        x (ndarray (n, )): Input vector
        w (ndarray (n, )): Weight Vector. Model parameter
        b (scalar): Bias. Model parameter
    Returns:
        p (scalar): Prediction (house price for this case)
    """
    p = 0
    n = x.shape[0] # we are expecting 1 dimentional array
    for i in range(n):
        p += w[i] * x[i]
    p += b
    return p

def predict_single_dot(x: np.ndarray, w: np.ndarray, b:float):
    """
    Predicts output for a training input of multiple linear regression using dot product
    Args:
        x (ndarray (n, )): Input vector
        w (ndarray (n, )): Weight Vector. Model parameter
        b (scalar): Bias. Model parameter
    Returns:
        p (scalar): Prediction (house price for this case)
    """
    p = np.dot(w, x) + b
    return p

In [39]:
x_vec = X_train[0]
f_wb = predict_single_loop(x_vec, w_init, b_init)
print(f_wb)
f_wb = predict_single_dot(x_vec, w_init, b_init)
print(f_wb)

459.9999976194083
459.99999761940825


# Computation of cost with multiple variables
Cost Function for Multiple variables:
$$ J(\vec{w}, b) = \frac{1}{2m}\sum_{i=1}^{i=m}\left(f_{\vec{w}, b}(\vec{x}^i) - y^i\right)^2$$

In [40]:
def compute_cost(X: np.ndarray, y: np.ndarray, w: np.ndarray, b: float):
    """
    Compute cost for whole training dataset
    Args:
        X (ndarray (m, n)): Training input matrix
        y (ndarray (m, )): Training output vector
        w (ndarray (n, )): Weights
        b (scalar): Bias. Model Parameter
    Returns:
        cost (scalar): Cost of the model
    """

    cost_i = 0
    cost = 0
    m = X.shape[0]
    for i in range(m):
        f_wb_i = np.dot(w, X[i]) + b
        cost_i = (f_wb_i - y[i])**2
        cost += cost_i
    cost = cost/(2*m)
    return cost


In [41]:
# Compute cost using pre-chosen parameters
cost = compute_cost(X_train, y_train, w_init, b_init)
print(f"Cost: {cost}")

Cost: 1.5578904880036537e-12


# Gradient Descent for Multiple Variables

\begin{equation}
\begin{align}
  & w_j = w_j - \alpha \frac{\partial}{\partial w_j}J(\vec{w}, b) \\
  & \frac{\partial}{\partial w_j}J(\vec{w}, b) = \frac{1}{m}\sum_{i=1}^{i=m}\left(f_{\vec{w}, b}(\vec{x}^i) - y^i\right)x^{i}_j \\
  & b = b - \alpha \frac{\partial}{\partial b}J(\vec{w}, b) \\
  & \frac{\partial}{\partial b} J(\vec{w}, b) = \frac{1}{m}\sum_{i=1}^{i=m}\left(f_{\vec{w}, b}(\vec{x}^i) - y^i\right)
  \end{align}
  \end{equation}

In [42]:
def compute_gradient(X: np.ndarray, y: np.ndarray, w: np.ndarray, b: float): 
    """
    Computes the gradient for linear regression. Gradient is the partial derivative term
    Args:
        X (ndarray (m, n)): Training input matrix
        y (ndarray (m, )): Training output vector
        w (ndarray (n, )): Model parameter, weight vector
        b (scalar): Model parameter, bias
    Returns:
        dj_dw (ndarray (n, )): Vector of derivative terms or gradient of weights
        dj_db (scalar): Gradient term of bias
    """

    m, n = X.shape
    dj_db = 0
    dj_dw_i = 0
    dj_dw = np.zeros(shape=(n, ))
    for i in range(m):
        err = (np.dot(X[i], w) + b) - y[i]
        for j in range(n):
            dj_dw[j] += err * X[i, j]
        dj_db +=  err
    dj_dw = dj_dw / m
    dj_db = dj_db / m
    return (dj_dw, dj_db)

In [43]:
tmp_dj_dw, tmp_dj_db = compute_gradient(X_train, y_train, w_init, b_init)
print(f"dj_dw: {tmp_dj_dw}")
print(f"dj_db: {tmp_dj_db}")

dj_dw: [-2.72623581e-03 -6.27197272e-06 -2.21745580e-06 -6.92403399e-05]
dj_db: -1.673925169143331e-06


In [45]:
def gradient_descent(X: np.ndarray, y:np.ndarray, w_in:np.ndarray, b_in:float, cost_function, gradient_function, alpha:float, num_iters:int):
    """
    Performes gradient descent for a multi linear regression. Updates the values of weights and bias.
    Args:
        X (ndarray (m, n))   : Training input matrix
        y (ndarray (m, ))    : Training output vector
        w_in (ndarray (n, )) : Initial weights vector
        b_in (scalar)        : Initial bias value
        cost_function        : Function to calculate cost of the whole dataset
        gradient_function    : Function to calculate gradient of a row
        alpha (float)        : The Learning Rate
        num_iters (int)      : Number of iterations to run gradient descent
    Returns:
        w (ndarray (n, ))    : Updated weights vector
        b (scalar)           : Updated bias vector   
    """
    J_history = []
    w = copy.deepcopy(w_in)
    b = b_in
    for i in range(num_iters):
        # calculation of the gradients
        dj_dw, dj_db = gradient_function(X, y, w, b)

        # update the parameters
        w = w - alpha * dj_dw       # Vector arithmatic on numpy arrays
        b = b - alpha * dj_db       # Normal Arighmatic

        cost = cost_function(X, y, w, b)
        J_history.append(cost)

        if i % math.ceil(num_iters/10) == 0:
            print(f"Iteration: {i}: Cost: {J_history[-1]:0.6f}")

    return w, b, J_history 

    

In [55]:
# Testing the gradient descent funtion

# Input Variables
initial_w = np.zeros_like(w_init)
initial_b = 0.00
iterations = 10000000
alpha = 5.0e-7

# Running the gradient_descent
w_final, b_final, J_hist = gradient_descent(X_train, y_train, initial_w, initial_b, compute_cost, compute_gradient, alpha, iterations)
print(f"w: {w_final}\nb:{b_final}")

Iteration: 0: Cost: 2529.462952
Iteration: 1000000: Cost: 455.074773
Iteration: 2000000: Cost: 359.074392
Iteration: 3000000: Cost: 283.325789
Iteration: 4000000: Cost: 223.556746
Iteration: 5000000: Cost: 176.396292
Iteration: 6000000: Cost: 139.184580
Iteration: 7000000: Cost: 109.822872
Iteration: 8000000: Cost: 86.655168
Iteration: 9000000: Cost: 68.374812
w: [  0.18173972  16.29622198 -45.62523857   0.76800375]
b:0.843349804325403


In [52]:
# Test the prediction accuracy
m, _ = X_train.shape
for i in range(m):
    print(f"Prediction {i}: {np.dot(X_train[i], w_final) + b_final} for {y_train[i]}")

Prediction 0: 426.18530497189204 for 460
Prediction 1: 286.1674720078562 for 232
Prediction 2: 171.46763087132317 for 178


In [56]:
# Test the prediction accuracy
m, _ = X_train.shape
for i in range(m):
    print(f"Prediction {i}: {np.dot(X_train[i], w_final) + b_final} for {y_train[i]}")

Prediction 0: 453.63976106696657 for 460
Prediction 1: 246.54513236734462 for 232
Prediction 2: 169.5329280675303 for 178
